In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd

## part 1

In [ ]:
def grad_n(r: np.array) -> np.array:
    grad_n = -2 / (sum(r**2) ** (3/2)) * r
    return grad_n

def n(r: np.array) -> float:
    n_ = 1 + 1 / np.linalg.norm(r)
    return n_


def step(r: np.array, v: np.array, s: np.array, l: np.array, i:int):
    # n_, grad_n = n(r)

    r[i+1] = r[i] + v[i] * s[i]
    v[i+1] = (grad_n(r[i]) * s[i] + n(r[i]) * v[i]) / n(r[i+1])
    s[i+1] =  np.linalg.norm(r[i+1] - r[i])

    l[i] = np.linalg.norm(r[i+1] - r[i]) / s[i]
    return r, v, s , l

def cut_all_zeros(n, *args):
    clean_args = []
    for arg in args:
        clean_args.append(args[:n])
    return tuple(clean_args)


x0 = 0
z0 = 1000
alpha0 = np.deg2rad(2) # deg 
vx0 = np.sin(alpha0)
vz0 = -np.cos(alpha0)
s0 = 1

epochs = 1000

r = np.zeros((epochs, 2))
v = np.zeros((epochs, 2))
s = np.zeros(epochs)
l = np.zeros(epochs)

r[0] = [x0, z0]
v[0] = [vx0, vz0]
s[0] = s0
s[1] = s0

for i in range(epochs -1):
    r, v, s, l = step(r, v, s, l, i)

    if r[i+1, 1] < 0:
        print(f'Epoch: {i}')
        r = r[:i+1]
        v = v[:i+1]
        s = s[:i+1]
        break


# df = pd.DataFrame({ 's': s, 'x': r[:, 0], 'z': r[:, 1] ,'vx': v[:, 0], 'vz': v[:, 1], 'l': l})

fig = px.line(x=r[:,0], y=r[:,1])

thory_line = np.array([
    r[0],
    r[0] + v[0] * 1000,
])

fig.add_scatter(x=thory_line[:,0], y=thory_line[:,1])

fig.show()

def solve(x0, z0, alpha0, s0, epochs):
    vx0 = np.sin(alpha0)
    vz0 = -np.cos(alpha0)

    r = np.zeros((epochs, 2))
    v = np.zeros((epochs, 2))
    s = np.zeros(epochs)
    l = np.zeros(epochs)

    r[0] = [x0, z0]
    v[0] = [vx0, vz0]
    s[0] = s0
    # s[1] = s0

    for i in range(epochs -1):
        r, v, s, l = step(r, v, s, l, i)

        if r[i+1, 1] < 0:
            print(f'Epoch: {i}')
            r, v, s = cut_all_zeros(epochs, r, v, s)
            break

    return r, v, s

### B

In [ ]:
# from thory calculation
def calculate_angle_shift(alpha0):
    vx0 = np.sin(alpha0)
    vz0 = -np.cos(alpha0)
    
    b = -z0 * vx0/ vz0 

    r, v, s = solve(x0=0, z0=1000, alpha0=alpha0, s0=1, epochs=1000)
    delta_alpha = np.arccos(np.dot(v[0], v[-1]) / (np.linalg.norm(v[0]) * np.linalg.norm(v[-1])))
    delta_alpha_theory = 2 / b
    error = np.abs(delta_alpha - delta_alpha_theory) / delta_alpha
    return error

angles = np.arange(1, 3, 0.1)
errors = []
for angle in angles:
    errors.append(calculate_angle_shift(angle))

px.scatter(x=angles, y=errors) 

## part 2